# Create Dataset

## Setup

In [1]:
library(tidyverse)
library(feather)
year <- lubridate::year

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
raw_data_file <- "../../../data/incidents_2019-08-11.feather"
largest_observations_file = "../../../preprocessed_data/largest_companies_2019-09-08.feather"
company_groups_file <- "../../../preprocessed_data/company_groups_2019-08-18.feather"
company_groups_file <- "../../../preprocessed_data/company_groups_2019-08-18.feather"
m_as_file <- "../../../preprocessed_data/m_as_2019-08-18.feather"

## Read Data

In [3]:
largest_observations <- read_feather(largest_observations_file)
head(largest_observations)

OPERATOR_ID,YEAR,NAME,TOTAL_MILES
<chr>,<dbl>,<chr>,<dbl>
31618,2018,ENTERPRISE PRODUCTS OPERATING LLC,8311.369
32109,2018,"ONEOK NGL PIPELINE, LLC",4756.610
31684,2005,PHILLIPS 66 PIPELINE LLC,4691.000
22610,2014,"MAGELLAN PIPELINE COMPANY, LP",4505.500
2552,2015,COLONIAL PIPELINE CO,4500.920
1845,2015,"BUCKEYE PARTNERS, LP",4298.290


In [4]:
raw_data <- read_feather(raw_data_file)
head(raw_data[, 1:20])

Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”

DATAFILE_AS_OF,SIGNIFICANT,SERIOUS,IPE,IA_IPE,OM_IPE,REPORT_NUMBER,SUPPLEMENTAL_NUMBER,REPORT_RECEIVED_DATE,REPORT_TYPE,OPERATOR_ID,NAME,OPERATOR_STREET_ADDRESS,OPERATOR_CITY_NAME,OPERATOR_STATE_ABBREVIATION,OPERATOR_POSTAL_CODE,LOCAL_DATETIME,IYEAR,LOCATION_LATITUDE,LOCATION_LONGITUDE
<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dttm>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dttm>,<dbl>,<dbl>,<dbl>
2019-07-31 04:37:40,NO,NO,NO,NO,NO,20100001,15751,2010-03-10 08:05:46,SUPPLEMENTAL FINAL,22610,"MAGELLAN PIPELINE COMPANY, LP","MAGELLAN MIDSTREAM PARTNERS, L.P.",TULSA,OK,74172,2010-02-16 02:42:00,2010,41.94352,-88.23353
2019-07-31 04:37:40,NO,NO,NO,NO,NO,20100002,19837,2010-03-16 13:43:54,SUPPLEMENTAL FINAL,31672,"CHAPARRAL ENERGY, LLC",701 CEDAR LAKE BLVD,OKLAHOMA CITY,OK,73114,2010-03-01 06:50:00,2010,37.10847,-100.80037
2019-07-31 04:37:40,YES,NO,NO,NO,NO,20100003,18021,2010-03-17 06:38:15,SUPPLEMENTAL FINAL,32035,LDH ENERGY PIPELINE L.P.,13430 NORTHWEST FREEWAY SUITE 1200,HOUSTON,TX,77040-6019,2010-02-22 05:38:00,2010,32.22471,-101.40440
2019-07-31 04:37:40,NO,NO,NO,NO,NO,20100004,19086,2010-03-18 09:18:48,SUPPLEMENTAL FINAL,1845,"BUCKEYE PARTNERS, LP",FIVE TEK PARK,BREINIGSVILLE,PA,18031,2010-02-19 01:50:00,2010,40.60860,-74.23990
2019-07-31 04:37:40,NO,NO,NO,NO,NO,20100005,17663,2010-03-18 10:26:48,SUPPLEMENTAL FINAL,300,"PLAINS PIPELINE, L.P.",333 CLAY STREET,HOUSTON,TX,772104648,2010-02-21 07:45:00,2010,31.13284,-101.18974
2019-07-31 04:37:40,NO,NO,NO,NO,NO,20100006,17891,2010-03-19 10:48:18,SUPPLEMENTAL FINAL,31684,CONOCOPHILLIPS,600 NORTH DAIRY ASHFORD,HOUSTON,TX,77079,2010-02-22 05:56:00,2010,47.71696,-117.35583


In [5]:
company_groups <- read_feather(company_groups_file)
glimpse(company_groups)

Observations: 29
Variables: 2
$ members <chr> "31618", "30829", "3445", "1845", "31371", "32147", "22830", …
$ name    <chr> "Enterprise Products (Group)", "Enterprise Products (Group)",…


In [6]:
company_groups <- read_feather(company_groups_file)
head(company_groups)
m_as <- read_feather(m_as_file)
head(m_as)

members,name
<chr>,<chr>
31618,Enterprise Products (Group)
30829,Enterprise Products (Group)
3445,Enterprise Products (Group)
1845,Buckeye (Group)
31371,Buckeye (Group)
32147,Marathon (Group)


members,name,start_year,end_year
<chr>,<chr>,<chr>,<chr>
Marathon (Group),Marathon (Group),2018,NA
Tesoro (Group),Marathon (Group),2018,NA
Sunoco (Group),Sunoco (Group),2017,NA
Energy Transfer (Group),Sunoco (Group),2017,NA
Kinder Morgan (Group),Kinder Morgan (Group),NA,2013
31720,Kinder Morgan (Group),NA,2013


## Define functions

In [7]:
get_latest_name <- function(dataset, id_col, name_col, time_col){
    quo_id_col <- enquo(id_col)
    quo_name_col <- enquo(name_col)
    quo_time_col <- enquo(time_col)
    
    dataset <- dataset %>%
        group_by(!! quo_id_col) %>%
        arrange(desc(!! quo_time_col)) %>%
        mutate(!! quo_name_col := first(!! quo_name_col))
    
    return(dataset)
}

In [8]:
create_dataset <- function(raw_data, variables, variable_names, operators, undesired_company_endings, commodity_names, causes, 
                           groups = company_groups$members, m_a = m_as$members
# Make sure you provide the right named vector that ensures that the respective column is renamed to the column expcted by the function for name, time, etc.
                           name_col = "Name", time_col = "Time", commodity_col = "Commodity", cause_col = "Cause", na_values = c("nan", "UNKNOWN")){
    dataset <- select(raw_data, variables)
    dataset <- rename(dataset, !!! variable_names)
    
    # For option to filter by sample.
    dataset$in_sample <- dataset$ID %in% operators
    
    # Clean (unify) company names.
    dataset <- get_latest_name(dataset, ID, Name, Time)
    
    # Various cleaning.
    dataset$Year = year(dataset[[time_col]])
    dataset[[name_col]] = str_remove_all(dataset[[name_col]], undesired_company_endings)
    dataset[[name_col]] = str_to_title(dataset[[name_col]])
    dataset[[commodity_col]] = recode(dataset[[commodity_col]], !!! commodity_names)
    dataset[[cause_col]] = recode(dataset[[cause_col]], !!! causes)
    # Weird method to replace values with NA cause naniar functions stopped working.
    for (i in colnames(dataset)){
        dataset[dataset[[i]] %in% na_values, i] <- NA
    }
    
    dataset$in_group <- dataset$ID %in% groups
    dataset$in_m_a <- dataset$ID %in% m_a
    dataset$Group <- dataset$in_group | dataset$in_m_a
    
    # Recode from yes/no to boolean.
    for (column in colnames(dataset)){
        if (all(unique(dataset[[column]]) %in% c("YES", "NO", NA))){
            dataset[[column]] <- dataset[[column]] == "YES"
        }
    }
    
    return(dataset)
}

## Run

In [9]:
variables <- c("OPERATOR_ID", "NAME", "LOCAL_DATETIME", "LOCATION_LATITUDE", "LOCATION_LONGITUDE", "SIGNIFICANT", "SERIOUS", "ON_OFF_SHORE",
               "IGNITE_IND", "EXPLODE_IND", "NUM_PUB_EVACUATED", "INJURE", "UNINTENTIONAL_RELEASE_BBLS", "COMMODITY_RELEASED_TYPE", 
               "PIPE_DIAMETER", "PIPE_WALL_THICKNESS", "OCEAN_SEAWATER_IND", "GROUNDWATER_CONTAM_IND", "DRINKING_WATER_CONTAM_IND",
               "HIGH_POP_IND", "USA_DRINKING_IND", "USA_ECOLOGICAL_IND", "TOTAL_COST", "NUM_EMPLOYEES_FAILED", "CAUSE")

variable_names <- c("ID" = "OPERATOR_ID", 
                    "Name" = "NAME", 
                    "Time" = "LOCAL_DATETIME", 
                    "Lat" = "LOCATION_LATITUDE", 
                    "Long" = "LOCATION_LONGITUDE", 
                    "Significant" = "SIGNIFICANT", 
                    "Serious" = "SERIOUS", 
                    "Offshore" = "ON_OFF_SHORE",
                    "Fire" = "IGNITE_IND", 
                    "Explosion" = "EXPLODE_IND", 
                    "Evacuated" = "NUM_PUB_EVACUATED", 
                    "Injuries" = "INJURE", 
                    "Barrels released" = "UNINTENTIONAL_RELEASE_BBLS",
                    "Commodity" = "COMMODITY_RELEASED_TYPE", 
                    "Pipe diameter" = "PIPE_DIAMETER", 
                    "Pipe thickness" = "PIPE_WALL_THICKNESS",
                    "Ocean contamination" = "OCEAN_SEAWATER_IND",
                    "Groundwater contamination" = "GROUNDWATER_CONTAM_IND", 
                    "Drinking water contamination" = "DRINKING_WATER_CONTAM_IND", 
                    "High population area" = "HIGH_POP_IND", 
                    "Water protection area" = "USA_DRINKING_IND", 
                    "Ecological site" = "USA_ECOLOGICAL_IND", 
                    "Cost" = "TOTAL_COST", 
                    "Alcohol test failures" = "NUM_EMPLOYEES_FAILED", 
                    "Cause" = "CAUSE")

commodity_names <- c("BIOFUEL / ALTERNATIVE FUEL(INCLUDING ETHANOL BLENDS)" = "Biofuel", 
                     "CO2 (CARBON DIOXIDE)" = "CO2",
                     "CRUDE OIL" = "Crude", 
                     "HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS A GAS AT AMBIENT CONDITIONS" = "Propane, Butane, etc.", 
                     "REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHICH IS A LIQUID AT AMBIENT CONDITIONS" = "Gasoline, Diesel, etc.")

causes <- c("CORROSION FAILURE" = "Corrosion", 
            "EQUIPMENT FAILURE" = "Equipment", 
            "EXCAVATION DAMAGE" = "Excavation", 
            "INCORRECT OPERATION" = "Incorrect_operation", 
            "MATERIAL FAILURE OF PIPE OR WELD" = "Pipe_or_weld", 
            "NATURAL FORCE DAMAGE" = "Natural_force", 
            "OTHER INCIDENT CAUSE" = "Other_incident", 
            "OTHER OUTSIDE FORCE DAMAGE" = "Other_force")

undesired_company_endings <- c(" L. P.| L.L.C.|, LLC|, L.P.| L.P.|, LP|LP|LLC|, LIMITED PARTNERSHIP|, A DIVISION OF EXXON MOBIL CORPORATION")

In [10]:
dataset <- create_dataset(raw_data=raw_data, variables=variables, variable_names=variable_names, 
                          operators=largest_observations$OPERATOR_ID, undesired_company_endings=undesired_company_endings, 
                          commodity_names=commodity_names, causes=causes)
head(dataset)

ERROR: Error in match(x, table, nomatch = 0L): 'match' requires vector arguments


In [ ]:
write_feather(dataset, paste0("../incident_dashboard/data/dataset_", Sys.Date(), ".feather"))
write_feather(company_groups, "../incident_dashboard/data/company_groups.feather")

In [ ]:
sum(dataset$in_sample)

In [ ]:
nrow(dataset)